In [1]:
from papyrus_scripts import matchRCSB

The goal of this notebook is to have an overview of the required input parameters to scrape the RCSB database and match those to ligands in the papyrus dataset.

The output is a seperate tsv file with the matched entries.

It is also possible to import the modules and run this code snippet in your own workflow.

In [2]:
# required
papyrus_inputfile  = '/home/willem/data/papyrus/'  # location of papyrus dataset

# optional
workdir            = '/home/willem/tmp'            # Working directory
json_infile        = None                          # Specify a RCSB_data .json file, avoids requering RCSB
json_outfile       = 'RCSB_data.json'              # Save the RCSB_data file under a different filename
papyrus_outputfile = 'Papyrus-RCSB_matched.tsv.gz' # provide a different filename for the RCSB match Papyrus entries
overwrite          = True                          # Set false to avoid overwriting the workdir

# put the user input in data
data = {
         'wd'        : workdir,
         'js_in'     : json_infile,
         'js_out'    : json_outfile,
         'p_in'      : papyrus_inputfile,
         'p_out'     : papyrus_outputfile,
         'overwrite' : overwrite
}

In [ ]:
matchRCSB.Init(data)

reading data from Papyrus sdf
[                                        ] | 0% Completed | 41.0s